## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf

In [ ]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')

In [2]:
import pandas as pd
import numpy as np

# For hypothesis testing
from scipy.stats import ttest_ind
from statsmodels.stats.power import ttest_power

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [3]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [4]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [5]:
data.drop(["date", "symbol"], axis=1, inplace=True)

In [6]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [7]:
# Divide 'Volume' by 1000000
data['volume'] = data['volume']/(1000*1000)

# Create sample of size 1000
data_s = data.iloc[0:1000]
#data_s = data.sample(n=1000, random_state=1)

data_s.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


In [8]:
# Check if the 1000 records represent the population

# Define funtion to do hypothesis testing
def nullHypothesisValidator(original, updated, serviceColumn, 
                            replaceWithOriginal_ifNullHypthesisRejected=True, p_val_threshold=0.05):
    print("***Checking Null Hypothesis for: ", serviceColumn)
    t_stat, p_val = ttest_ind(original[serviceColumn], updated[serviceColumn])
    #print("t_stat: ", t_stat)
    print("p_val: ", p_val)
    if replaceWithOriginal_ifNullHypthesisRejected:
        if p_val < p_val_threshold:
            updated[serviceColumn] = original[serviceColumn]
            print("Reintroduced column with original values for feature ***", serviceColumn, "'***")
    return


#---***---


for col_e in data_s.columns:
    nullHypothesisValidator(data, data_s, col_e, replaceWithOriginal_ifNullHypthesisRejected=False)



***Checking Null Hypothesis for:  open
p_val:  0.1142563613745002
***Checking Null Hypothesis for:  close
p_val:  0.12318076107154532
***Checking Null Hypothesis for:  low
p_val:  0.11442174960818424
***Checking Null Hypothesis for:  high
p_val:  0.1141680349671109
***Checking Null Hypothesis for:  volume
p_val:  0.7986125001445838


Null hypothesis is accepted for all columns

### Divide the data into train and test sets

In [17]:
X = data_s.drop("volume", axis=1)
y = data_s['volume']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=7)

#### Convert Training and Test Data to numpy float32 arrays


In [18]:
for col_e in X.columns:
    X_train[col_e] = X_train[col_e].astype(np.float32)
    X_test[col_e] = X_test[col_e].astype(np.float32)

y_train.volume = y_train.astype(np.float32)
y_test.volume = y_test.astype(np.float32)

C:\installationDirectory\PythonInstallation\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\installationDirectory\PythonInstallation\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [19]:
# standardize values in each feature
standardizer = preprocessing.Normalizer()

# 
X_train = pd.DataFrame(standardizer.fit_transform(X_train), columns=X.columns)
X_test = pd.DataFrame(standardizer.fit_transform(X_test), columns=X.columns)

# Convert to array of type float32 (tensorflow understands array)
X_train = np.array(X_train).astype("float32")
X_test = np.array(X_test).astype("float32")
y_train = np.array(y_train).astype("float32")
y_test = np.array(y_train).astype("float32")

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [20]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [21]:
# def prediction(x_in, w_in, b_in):
#     xw_matrix_multiplication_temp = tf.matmul(x_in, w_in)
#     y_out = tf.add(xw_matrix_multiplication_temp, b_in)
#     return y_out

def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [22]:
# def loss(y_actual_in, y_pred_in):
#     diff_temp = y_actual_in - y_pred_in
#     sqr_temp = tf.square(diff_temp)
#     avg_temp = tf.reduce_mean(sqr_temp)
#     return avg_temp

def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [23]:
# def train(x_in, y_in, w_in, b_in, learning_rate_in=0.01):
#     with tf.GradientTape() as t_temp:     # Record to calculate loss
#         t_temp.watch([w_in, b_in])
        
#         y_prediction_temp = prediction(x_in, w_in, b_in)
#         loss_temp = loss(y_in, y_prediction_temp)
        
#     dw_temp, db_temp = t_temp.gradient(loss_temp, [w_in, b_in])     # Calculate gradient
    
#     w_out = w - learning_rate_in*dw_temp
#     b_out = w - learning_rate_in*db_temp
    
#     return w_out, b_out     # Return new values for weight and bias

def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [28]:
# Convert DataFrame to tensor Dataset
# tf_target = data.pop("volume")
# tf_data = tf.data.Dataset.from_tensor_slices((data.values, tf_target))

#(train_x, train_y),(_,_) = tf.keras.datasets.boston_housing.load_data(test_split=0)
# (X_train, y_train),(X_test,y_test) = tf_data(test_split=0)

for i in range(100):
    w, b = train(X_train, y_train, w, b, learning_rate=0.01)
    if i%5 == 0:
        print("Loss in iteration ", i, "is: ", loss(y_train, prediction(X_train, w, b)).numpy())

Loss in iteration  0 is:  244.7527
Loss in iteration  5 is:  244.7527
Loss in iteration  10 is:  244.7527
Loss in iteration  15 is:  244.7527
Loss in iteration  20 is:  244.7527
Loss in iteration  25 is:  244.7527
Loss in iteration  30 is:  244.7527
Loss in iteration  35 is:  244.7527
Loss in iteration  40 is:  244.7527
Loss in iteration  45 is:  244.7527
Loss in iteration  50 is:  244.7527
Loss in iteration  55 is:  244.7527
Loss in iteration  60 is:  244.7527
Loss in iteration  65 is:  244.7527
Loss in iteration  70 is:  244.7527
Loss in iteration  75 is:  244.7527
Loss in iteration  80 is:  244.7527
Loss in iteration  85 is:  244.7527
Loss in iteration  90 is:  244.7527
Loss in iteration  95 is:  244.7527


### Get the shapes and values of W and b

In [32]:
print("w values: ", w)
print("w shape: ", w.shape)
# print("\n\n***\n\n")

w values:  tf.Tensor(
[[1.3466516]
 [1.3512989]
 [1.3326494]
 [1.3629096]], shape=(4, 1), dtype=float32)
w shape:  (4, 1)


In [33]:
print("b values: ", b)
print("b shape: ", b.shape)

b values:  tf.Tensor([2.6969109], shape=(1,), dtype=float32)
b shape:  (1,)


### Model Prediction on 1st Examples in Test Dataset

In [35]:
y_pred = prediction(X_test, w, b)
y_pred

<tf.Tensor: id=16262, shape=(200, 1), dtype=float32, numpy=
array([[5.3935823],
       [5.3937335],
       [5.393728 ],
       [5.3936653],
       [5.3935575],
       [5.3937325],
       [5.3937154],
       [5.3937435],
       [5.393719 ],
       [5.3937364],
       [5.3937416],
       [5.39374  ],
       [5.39375  ],
       [5.3937273],
       [5.3936534],
       [5.3937426],
       [5.3936543],
       [5.393712 ],
       [5.393426 ],
       [5.393717 ],
       [5.3937364],
       [5.3937325],
       [5.393653 ],
       [5.393714 ],
       [5.393676 ],
       [5.393678 ],
       [5.3934507],
       [5.3937225],
       [5.393339 ],
       [5.3934035],
       [5.393719 ],
       [5.3935795],
       [5.3937373],
       [5.393663 ],
       [5.3936725],
       [5.393741 ],
       [5.393735 ],
       [5.3937216],
       [5.3937235],
       [5.3937216],
       [5.393743 ],
       [5.3937144],
       [5.3937273],
       [5.393718 ],
       [5.3936934],
       [5.393657 ],
       [5.3937025],


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [90]:
df_iris = pd.read_csv("Iris.csv")
df_iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [91]:
# Drop Id
df_iris.drop("Id", axis=1, inplace=True)

df_iris = pd.get_dummies(df_iris, columns=["Species"])
#df_iris["Species"] = pd.get_dummies(df_iris["Species"])
df_iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [98]:
X = df_iris[df_iris.columns[0:4]]
Y = df_iris[df_iris.columns[4:7]]

# X = np.array(X).astype("float32")
# Y = np.array(Y).astype("float32")

# Divide into test and train set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=7)

# Standardize
X_train = pd.DataFrame(standardizer.fit_transform(X_train), columns=X.columns)
X_test = pd.DataFrame(standardizer.fit_transform(X_test), columns=X.columns)

# # Convert to array of type float32 (tensorflow understands array)
# X_train = np.array(X_train).astype("float32")
# X_test = np.array(X_test).astype("float32")
# y_train = np.array(y_train).astype("float32")
# y_test = np.array(y_train).astype("float32")

In [93]:
# trainY = tf.keras.utils.to_categorical(y_train, num_classes=3)
# testY = tf.keras.utils.to_categorical(y_test, num_classes=3)
# trainX = X_train
# testX = X_test

# print(trainY.shape)
# print('First 2 examples now are: ', trainY[0:2])

In [94]:
# print(y_train.shape)
# print('First 2 examples now are: ', y_train[0:2])

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [100]:
# model = tf.keras.models.Sequential()

# model.add(tf.keras.layers.BatchNormalization())

# model.add(tf.keras.layers.Dense(units=10, activation="Softmax"))
# model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=["accuracy"])

#---***---

#Initialize Sequential model
model = tf.keras.models.Sequential()

# #Reshape data from 2D to 1D -> 28x28 to 784
# model.add(tf.keras.layers.Reshape((784,),input_shape=(28,28,)))

# #Normalize the data
# model.add(tf.keras.layers.BatchNormalization())

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, input_shape=(4,), activation='softmax'))

#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

### Model Training 

In [101]:
# model.fit(X_train, y_train, validation_data=(X_test, y_test),
#          epochs=10,
#          batch_size=50)

model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          epochs=100,
          batch_size=trainX.shape[0])

Train on 120 samples, validate on 30 samples
Epoch 1/100
120/120 [==============================] - 1s 6ms/sample - loss: 1.1693 - accuracy: 0.3250 - val_loss: 1.0828 - val_accuracy: 0.3667
Epoch 2/100
120/120 [==============================] - 0s 83us/sample - loss: 1.1680 - accuracy: 0.3250 - val_loss: 1.0821 - val_accuracy: 0.3667
Epoch 3/100
120/120 [==============================] - 0s 75us/sample - loss: 1.1666 - accuracy: 0.3250 - val_loss: 1.0813 - val_accuracy: 0.3667
Epoch 4/100
120/120 [==============================] - 0s 75us/sample - loss: 1.1653 - accuracy: 0.3250 - val_loss: 1.0806 - val_accuracy: 0.3667
Epoch 5/100
120/120 [==============================] - 0s 83us/sample - loss: 1.1640 - accuracy: 0.3250 - val_loss: 1.0799 - val_accuracy: 0.3667
Epoch 6/100
120/120 [==============================] - 0s 75us/sample - loss: 1.1628 - accuracy: 0.3250 - val_loss: 1.0792 - val_accuracy: 0.3667
Epoch 7/100
120/120 [==============================] - 0s 75us/sample - loss: 1.

Epoch 56/100
120/120 [==============================] - 0s 75us/sample - loss: 1.1123 - accuracy: 0.3250 - val_loss: 1.0553 - val_accuracy: 0.3667
Epoch 57/100
120/120 [==============================] - 0s 75us/sample - loss: 1.1115 - accuracy: 0.3250 - val_loss: 1.0549 - val_accuracy: 0.3667
Epoch 58/100
120/120 [==============================] - 0s 75us/sample - loss: 1.1108 - accuracy: 0.3250 - val_loss: 1.0546 - val_accuracy: 0.3667
Epoch 59/100
120/120 [==============================] - 0s 75us/sample - loss: 1.1100 - accuracy: 0.3250 - val_loss: 1.0543 - val_accuracy: 0.3667
Epoch 60/100
120/120 [==============================] - 0s 83us/sample - loss: 1.1092 - accuracy: 0.3250 - val_loss: 1.0540 - val_accuracy: 0.3667
Epoch 61/100
120/120 [==============================] - 0s 83us/sample - loss: 1.1085 - accuracy: 0.3250 - val_loss: 1.0537 - val_accuracy: 0.3667
Epoch 62/100
120/120 [==============================] - 0s 92us/sample - loss: 1.1077 - accuracy: 0.3250 - val_loss: 1

### Model Prediction

In [103]:
y_pred = model.predict(X_test)
y_pred

array([[0.2303809 , 0.3411309 , 0.42848822],
       [0.23283848, 0.33846807, 0.4286935 ],
       [0.27740502, 0.33316785, 0.38942713],
       [0.23390073, 0.33866453, 0.42743474],
       [0.22729212, 0.34423834, 0.42846948],
       [0.2735263 , 0.33397734, 0.39249635],
       [0.23910977, 0.33653402, 0.42435613],
       [0.23714174, 0.33648732, 0.42637092],
       [0.27404436, 0.32655996, 0.3993957 ],
       [0.23832524, 0.33913556, 0.42253917],
       [0.23568577, 0.34019977, 0.42411456],
       [0.2354448 , 0.3409976 , 0.42355755],
       [0.28020582, 0.32496914, 0.394825  ],
       [0.22823654, 0.3480743 , 0.42368916],
       [0.278516  , 0.32937086, 0.39211312],
       [0.23149824, 0.3434623 , 0.42503944],
       [0.23238234, 0.3368419 , 0.4307758 ],
       [0.23175794, 0.3381709 , 0.4300712 ],
       [0.27602196, 0.32701185, 0.39696622],
       [0.27747768, 0.32461005, 0.39791223],
       [0.24102643, 0.3367765 , 0.42219704],
       [0.22920965, 0.34756738, 0.42322296],
       [0.

### Save the Model

In [104]:
model.save('mnist_lc.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?